In [85]:
# http://www.shirpeled.com/2018/10/a-hands-on-tutorial-for-zero-knowledge_4.html

import random

def get_witness(problem, assignment):
    """
    Given an instance of a partition problem via a list of numbers (the problem) and a list of
    (-1, 1), we say that the assignment satisfies the problem if their dot product is 0.
    """
    assert len(problem) == len(assignment)

    # partial sum
    s = 0
    side_obfuscator = 1 if random.random() > 0.5 else -1
    witness = [s]
    
    for num, side in zip(problem, assignment):
        assert side == 1 or side == -1
        s += side * num * side_obfuscator
        witness.append(s)
    
    assert s == 0
    shift = random.randint(0, max(0, max(problem)))
    witness = [x + shift for x in witness]
        
    return witness

### Test
problem = [4, 11, 8, 1]
assignment = [1, -1, 1, -1]

w = get_witness(problem, assignment)
n = len(problem)

print("problem", problem)
print("witness", w)

### Checks
assert w[0] == w[n]

for i in range(n):
    assert abs(w[i + 1] - w[i]) == abs(problem[i])


problem [4, 11, 8, 1]
witness [4, 8, -3, 5, 4]


In [9]:
import hashlib
from math import log2, ceil

def hash_string(s):
    return hashlib.sha256(s.encode()).hexdigest()

class MerkleTree:
    """
    A naive Merkle tree implementation using SHA256
    """
    def __init__(self, data):
        self.data = data
        next_pow_of_2 = int(2**ceil(log2(len(data))))
        self.data.extend([0] * (next_pow_of_2 - len(data)))
        self.tree = ["" for x in self.data] + \
                    [hash_string(str(x)) for x in self.data]
        for i in range(len(self.data) - 1, 0, -1):
            self.tree[i] = hash_string(self.tree[i * 2] + self.tree[i * 2 + 1])

    def get_root(self):
        return self.tree[1]

    def get_val_and_path(self, i):
        val = self.data[i]
        auth_path = []
        i = i + len(self.data)
        while i > 1:
            auth_path += [self.tree[i ^ 1]]
            i = i // 2
        return val, auth_path

def verify(root, data_size, i, value, path):
    cur = hash_string(str(value))
    tree_node_id = i + int(2**ceil(log2(data_size)))
    for sibling in path:
        assert tree_node_id > 1
        if tree_node_id % 2 == 0:
            cur = hash_string(cur + sibling)
        else:
            cur = hash_string(sibling + cur)
        tree_node_id = tree_node_id // 2
    assert tree_node_id == 1
    return root == cur

data = ["Yes", "Sir", "I Can", "Boogie!"]
merkle_tree = MerkleTree(data)

root = merkle_tree.get_root()
(val, path) = merkle_tree.get_val_and_path(1)

verify(root, len(data), 1, val, path)

True